# Create Model Runs

In [ ]:
import os
import sys
import shutil
import subprocess
from tempfile import mkstemp
from shutil import move, copymode
from os import fdopen, remove
import time

# Copy a directory
optionBaseDir='/glade/work/currierw/icar/crb/' # configuration
outBaseDir='/glade/scratch/currierw/icar/crb/' # output

exampleDir='tmp/2000-09_2001_05'

oldModelRunDir='eraiLongNeg1mp/erai2000-09_2001_05'

# New Model Run Base
newModelRunBase = optionBaseDir+'eraiLongNeg3mpNew/'
newModelRunOut  = outBaseDir+'eraiLongNeg3mpNew/'

# Dates to run the model
startDates = ['2000-09','2001-03','2001-09','2002-03','2002-09','2003-03','2003-09','2004-03']
endDates   = ['2001_04','2001_10','2002_04','2002_10','2003_04','2003_10','2004_04','2004_10']
endDates2  = ['2001-04','2001-10','2002-04','2002-10','2003-04','2003-10','2004-04','2004-10']


def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    #Copy the file permissions from the old file to the new file
    copymode(file_path, abs_path)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)


for i in range(0,len(startDates)):
    # Iterate over dates
    newModelRunDir=startDates[i]+'_'+endDates[i]
    
    # Copy a directory to create
    os.system('mkdir '+newModelRunBase)
    os.system('cp -r ' + optionBaseDir+oldModelRunDir +' '+newModelRunBase+newModelRunDir)
    print('Copied directory: ' + optionBaseDir+oldModelRunDir +' to the following: ' + newModelRunBase+newModelRunDir)
    
    # Make a New Output Directory to Store Model Run Output
    os.system('mkdir -p '+newModelRunOut+newModelRunDir)
    print('Made new output directory: ' + newModelRunOut+newModelRunDir)
    
    # Create a new symlink
    print('Creating a new symlink')
    os.system('ln -sfn '+newModelRunOut+newModelRunDir+' '+newModelRunBase+newModelRunDir+'/output')
    
    ######### Change Configuration File ######################
    # Grep the start date, remove trailing \n, insert new time
    sDates = subprocess.check_output('grep  start_date= '+newModelRunBase+newModelRunDir+'/base_run_options.nml', shell=True)
    tmp=str.split(str(sDates))[1].strip('\\n')
    replaceString='start_date="'+startDates[i]+'-01_00:00:00"'+'\n'
    replace(newModelRunBase+newModelRunDir+'/base_run_options.nml',tmp,replaceString)

    # Grep the end date, remove trailing \n, insert new time
    sDates = subprocess.check_output('grep end_date '+newModelRunBase+newModelRunDir+'/base_run_options.nml', shell=True)
    # print(str(sDates))
    tmp=str.split(str(sDates))[1]+' '+str.split(str(sDates))[2]+' '+str.split(str(sDates))[3][:-3]
    # print('tmp           = '+tmp)
    replaceString='end_date="'+endDates2[i]+'-01_00:00:00"'+'\n'
    # print('replaceString = '+replaceString)
    replace(newModelRunBase+newModelRunDir+'/base_run_options.nml',tmp,replaceString)
    ##########################################################

#     # CD into the directory we created and manipulated
#     os.chdir(newModelRunBase+newModelRunDir)
#     print("Current Working Directory: " , os.getcwd())
    
#     # Execute the pbs/.sh batch script
#     print("Submitting File to Cheyenne")
#     os.system('qsub base_run_batch_submit.sh')
    
#     time.sleep(3)
    

# CCSM4
* Takes a base directory with a base configuration file. Changes the following
* symlink to forcing directory
* creates new directories to stagger the model runs in time
    * adjust the time stamps within the configuration file
    * creates sub directories for the output of each staggered model run
    * updates the symlnks for the staggered output directories

In [3]:
import os
import sys
import shutil
import subprocess
from tempfile import mkstemp
from shutil import move, copymode
from os import fdopen, remove
import time

# Copy a directory
optionBaseDir='/glade/work/currierw/icar/crb/' # configuration
outBaseDir='/glade/scratch/currierw/icar/crb/' # output

exampleDir='tmp/2000-09_2001_05'

# New Model Run Base
newModelRunBase = optionBaseDir+'CCSM4testNeg3Mp/'
newModelRunOut  = outBaseDir+'CCSM4testNeg3Mp/'

# Dates to run the model
startDates = ['2000-09','2001-03','2001-09','2002-03','2002-09','2003-03','2003-09','2004-03']
endDates   = ['2001_04','2001_10','2002_04','2002_10','2003_04','2003_10','2004_04','2004_10'] # different seperator -/_
endDates2  = ['2001-04','2001-10','2002-04','2002-10','2003-04','2003-10','2004-04','2004-10'] # different seperator -/_

# Forcing Dir
forcingDir='/glade/scratch/currierw/CCSM4/historical/forcing'

######## Dont change below #########

def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    #Copy the file permissions from the old file to the new file
    copymode(file_path, abs_path)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)


for i in range(0,len(startDates)):
    # Iterate over dates
    newModelRunDir=startDates[i]+'_'+endDates[i]
    
    # Copy a directory to create
    os.system('mkdir '+newModelRunBase)
    os.system('cp -r ' + optionBaseDir+exampleDir +' '+newModelRunBase+newModelRunDir)
    print('Copied directory: ' + optionBaseDir+exampleDir +' to the following: ' + newModelRunBase+newModelRunDir)
    
    # Make a New Output Directory to Store Model Run Output
    os.system('mkdir -p '+newModelRunOut+newModelRunDir)
    print('Made new output directory: ' + newModelRunOut+newModelRunDir)
    
    # Create a new symlink - for output directory
    print('Creating new symlinks')
    os.system('ln -sfn '+newModelRunOut+newModelRunDir+' '+newModelRunBase+newModelRunDir+'/output')
    
    # Create a new symlink - for forcing directory
    os.system('ln -sfn '+forcingDir+' '+newModelRunBase+newModelRunDir+'/forcing')
    
    ######### Change Configuration File ######################
    # Grep the start date, remove trailing \n, insert new time
    sDates = subprocess.check_output('grep  start_date= '+newModelRunBase+newModelRunDir+'/base_run_options.nml', shell=True)
    tmp=str.split(str(sDates))[1].strip('\\n')
    replaceString='start_date="'+startDates[i]+'-01_00:00:00"'+'\n'
    replace(newModelRunBase+newModelRunDir+'/base_run_options.nml',tmp,replaceString)

    # Grep the end date, remove trailing \n, insert new time
    sDates = subprocess.check_output('grep end_date '+newModelRunBase+newModelRunDir+'/base_run_options.nml', shell=True)
    # print(str(sDates))
    tmp=str.split(str(sDates))[1]+' '+str.split(str(sDates))[2]+' '+str.split(str(sDates))[3][:-3]
    # print('tmp           = '+tmp)
    replaceString='end_date="'+endDates2[i]+'-01_00:00:00"'+'\n'
    # print('replaceString = '+replaceString)
    replace(newModelRunBase+newModelRunDir+'/base_run_options.nml',tmp,replaceString)
    ##########################################################

Copied directory: /glade/work/currierw/icar/crb/tmp/2000-09_2001_05 to the following: /glade/work/currierw/icar/crb/CCSM4testNeg3Mp/2000-09_2001_04
Made new output directory: /glade/scratch/currierw/icar/crb/CCSM4testNeg3Mp/2000-09_2001_04
Creating new symlinks
Copied directory: /glade/work/currierw/icar/crb/tmp/2000-09_2001_05 to the following: /glade/work/currierw/icar/crb/CCSM4testNeg3Mp/2001-03_2001_10
Made new output directory: /glade/scratch/currierw/icar/crb/CCSM4testNeg3Mp/2001-03_2001_10
Creating new symlinks
Copied directory: /glade/work/currierw/icar/crb/tmp/2000-09_2001_05 to the following: /glade/work/currierw/icar/crb/CCSM4testNeg3Mp/2001-09_2002_04
Made new output directory: /glade/scratch/currierw/icar/crb/CCSM4testNeg3Mp/2001-09_2002_04
Creating new symlinks
Copied directory: /glade/work/currierw/icar/crb/tmp/2000-09_2001_05 to the following: /glade/work/currierw/icar/crb/CCSM4testNeg3Mp/2002-03_2002_10
Made new output directory: /glade/scratch/currierw/icar/crb/CCSM4t

# CNRM-CM5

In [1]:
import os
import sys
import shutil
import subprocess
from tempfile import mkstemp
from shutil import move, copymode
from os import fdopen, remove
import time

# Copy a directory
optionBaseDir='/glade/work/currierw/icar/crb/' # configuration
outBaseDir='/glade/scratch/currierw/icar/crb/' # output

exampleDir='tmp/2000-09_2001_05'

# New Model Run Base
newModelRunBase = optionBaseDir+'CNRM-CM5-testNeg1Mp/'
newModelRunOut  = outBaseDir+'CNRM-CM5-testNeg1Mp/'

# Dates to run the model
startDates = ['2000-09','2001-03','2001-09','2002-03','2002-09','2003-03','2003-09','2004-03']
endDates   = ['2001_04','2001_10','2002_04','2002_10','2003_04','2003_10','2004_04','2004_10'] # different seperator -/_
endDates2  = ['2001-04','2001-10','2002-04','2002-10','2003-04','2003-10','2004-04','2004-10'] # different seperator -/_

# Forcing Dir
forcingDir='/glade/scratch/currierw/CNRM-CM5/historical/forcing'

######## Dont change below #########

def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    #Copy the file permissions from the old file to the new file
    copymode(file_path, abs_path)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)


for i in range(0,len(startDates)):
    # Iterate over dates
    newModelRunDir=startDates[i]+'_'+endDates[i]
    
    # Copy a directory to create
    os.system('mkdir '+newModelRunBase)
    os.system('cp -r ' + optionBaseDir+exampleDir +' '+newModelRunBase+newModelRunDir)
    print('Copied directory: ' + optionBaseDir+exampleDir +' to the following: ' + newModelRunBase+newModelRunDir)
    
    # Make a New Output Directory to Store Model Run Output
    os.system('mkdir -p '+newModelRunOut+newModelRunDir)
    print('Made new output directory: ' + newModelRunOut+newModelRunDir)
    
    # Create a new symlink - for output directory
    print('Creating new symlinks')
    os.system('ln -sfn '+newModelRunOut+newModelRunDir+' '+newModelRunBase+newModelRunDir+'/output')
    
    # Create a new symlink - for forcing directory
    os.system('ln -sfn '+forcingDir+' '+newModelRunBase+newModelRunDir+'/forcing')
    
    ######### Change Configuration File ######################
    # Grep the start date, remove trailing \n, insert new time
    sDates = subprocess.check_output('grep  start_date= '+newModelRunBase+newModelRunDir+'/base_run_options.nml', shell=True)
    tmp=str.split(str(sDates))[1].strip('\\n')
    replaceString='start_date="'+startDates[i]+'-01_00:00:00"'+'\n'
    replace(newModelRunBase+newModelRunDir+'/base_run_options.nml',tmp,replaceString)

    # Grep the end date, remove trailing \n, insert new time
    sDates = subprocess.check_output('grep end_date '+newModelRunBase+newModelRunDir+'/base_run_options.nml', shell=True)
    # print(str(sDates))
    tmp=str.split(str(sDates))[1]+' '+str.split(str(sDates))[2]+' '+str.split(str(sDates))[3][:-3]
    # print('tmp           = '+tmp)
    replaceString='end_date="'+endDates2[i]+'-01_00:00:00"'+'\n'
    # print('replaceString = '+replaceString)
    replace(newModelRunBase+newModelRunDir+'/base_run_options.nml',tmp,replaceString)
    ##########################################################

Copied directory: /glade/work/currierw/icar/crb/tmp/2000-09_2001_05 to the following: /glade/work/currierw/icar/crb/CNRM-CM5-testNeg1Mp/2000-09_2001_04
Made new output directory: /glade/scratch/currierw/icar/crb/CNRM-CM5-testNeg1Mp/2000-09_2001_04
Creating new symlinks
Copied directory: /glade/work/currierw/icar/crb/tmp/2000-09_2001_05 to the following: /glade/work/currierw/icar/crb/CNRM-CM5-testNeg1Mp/2001-03_2001_10
Made new output directory: /glade/scratch/currierw/icar/crb/CNRM-CM5-testNeg1Mp/2001-03_2001_10
Creating new symlinks
Copied directory: /glade/work/currierw/icar/crb/tmp/2000-09_2001_05 to the following: /glade/work/currierw/icar/crb/CNRM-CM5-testNeg1Mp/2001-09_2002_04
Made new output directory: /glade/scratch/currierw/icar/crb/CNRM-CM5-testNeg1Mp/2001-09_2002_04
Creating new symlinks
Copied directory: /glade/work/currierw/icar/crb/tmp/2000-09_2001_05 to the following: /glade/work/currierw/icar/crb/CNRM-CM5-testNeg1Mp/2002-03_2002_10
Made new output directory: /glade/scra